# A4MD: Validation of conformational space - Early termination of trajectories


---

## Overview

Through this notebook we are able to validate that the early termination of our framework A4MD covers the conformational space as the full simulation.
This notebook also provides visual representatations that helps to analyze the trimmed trajectories in comparison with the full simulation.

We are using the FS peptide system  (Ace-A 5(AAARA) 3A-NME) and its trajectories collected using Summit supercomputer. We use the trajectories of the full simulation and also trajectories with early termination using collective variables (CVs) such as Largest Eigen Value (LEV) and Effective Sample Size (ESS)

---

The following two cells prepares the environment for processing and visualizing the trajectories by importing various crucial libraries for the execution.

In [ ]:
# This code allows to reload the code in the notebook without restarting the kernel. Since we are using an external file with functions,
# this automatically loads the new modifications of that file into the notebook.
%load_ext autoreload
%autoreload 2

In [ ]:
# Importing libraries needed for the execution of the notebook
import pyemma.coordinates as coor
import mdtraj as md
from pyemma.util.contexts import settings
import numpy as np
import matplotlib as mpl
import pyemma
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import os
import JPCCCK_utils_2_reviews as jpccck
from IPython.display import display
import dataframe_image as dfi
import glob
from download_data import download_xtc_files

import pkg_resources
print("PyEmma version: ", pkg_resources.get_distribution("pyemma").version)

# You have have successfully prepared your environment.
print("You have successfully prepared your environment.")

## Downloading data
Make sure you have the full trajectories. The next cell will download the full trajectories from Dataverse. 

https://doi.org/10.7910/DVN/ML5607

Before running the notebook, also make sure you have generated the trimmed trajectories that you want to use for the comparison. The generation of these trajectories can be done by using [our bash script](./execute.sh)

In [ ]:
download_xtc_files()

## Define parameters
This cell defines some variables we need to load the trajectories, analize them and also, save frames on disk.

**NOTE:** Make sure you define the paths to your files.


In [ ]:
stride = 1  # Stride for loading trajectory
selection = (
    "protein"  # Used for the RMSD comparison. This option can be "protein" or "all"
)
input_dirs = [
    "./files/xtc_files/"
]  # Path fot the full trajectories of the simulation. Each folder containes 20 trajectories
trimmed_trajectory_1 = "./results/trajectories/<Add_your_1_early_terminated_trajectory>.xtc"  # Path for the trimmed trajectories of the simulation 
trimmed_trajectory_2 = "./results/trajectories/<Add_your_2_early_terminated_trajectory>.xtc"  # Path for the trimmed trajectories of the simulation 

top_file = "./files/xtc_files/boxed.pdb"  # Topology file for the trajectories
nstates = 6  # Number of states for the MSM model and the PCCA+
SAVE_FRAMES = True  # If True, the frames will be saved in the folder frames_closest
SAVE_PLOTS = True # If True, the plots will be saved in the folder plots
frames_closest_folder = "./files/frames/"  # Folder where the frames will be saved
dist_cmap = "nipy_spectral"  # Color map for the energy plots
size = 20  # Size of the point in the plots


Using this notebook you can generate the comparison of the Full trajectory with other two early-terminated trajectories.

Use the cell below to define the titles you want to have for every plot

In [ ]:
#titles
full_title = "Full" # Define
trimmed_1_title = "ESS - 18%" # Define
trimmed_2_title = "ESS - 44%" # Define

plots_1_name = trimmed_1_title.replace(" ","_")
plots_2_name = trimmed_2_title.replace(" ","_")
plots_names= plots_1_name+"_"+plots_2_name
plot_output_folder = "./files/plots"
print("Plots LEV name: ", plots_names)

## Loading Full trajectory

We start by creating a variable that contains every path of the full (40) trajectories

In [ ]:
trajs = glob.glob(input_dirs + "/*.xtc")

We load the trajectories as an MDtraj object. For this we use the previous variable that contains every path of the trajectories and the topological file.

In [ ]:
mdtrajectories = md.load(trajs, top=top_file)

We finally load the backbone torsions of the topological file and add them into memory so we have all the trajectories with their correspondent features

In [ ]:
feat = coor.featurizer(top_file) 
feat.add_backbone_torsions()
features_ref = feat.transform(mdtrajectories)

## Calculating Tica

To analyze the conformational space, we first create its 2D representation by applying a time-lagged Independent
Component Analysis (tICA) decomposition method to the full trajectories. We choose a time lag of 5000.

To do this task we use Pyemma's libraries. 



In [ ]:
with settings(show_progress_bars=True):
    tica_obj = coor.tica(features_ref, lag=5000, stride=stride)

We load the data from our tICA object using the get_output() function, which maps all input data and returns it as an array or list of arrays

In [ ]:
tica_outpu = tica_obj.get_output()

Finally we concatenate the all the values of the tICA space in a single variable.

In [ ]:
tica_concatenated_full = np.concatenate(tica_outpu)

## Clustering
To discretize the full space of the tICA space we use k-means clustering.

From now, we use only the first two tICA components which reflect the two slowest degrees of freedom.

In [ ]:
clust = coor.cluster_kmeans(
    tica_concatenated_full[:,:2], k=200, stride=stride, fixed_seed=True #, metric="minRMSD"
)
dtrajs = clust.dtrajs # Discretized trajectories
cc_x = clust.clustercenters[:, 0] # Cluster centers of the first tICA component
cc_y = clust.clustercenters[:, 1] # Cluster centers of the second tICA component
dtrajs_concatenated = np.concatenate(dtrajs) # Concatenated discretized trajectories

## Loading Trimmed trajectories

To do the comparison we need to load the trajectories that have been early terminated with the use of our A4MD framework. 

To cut the trajectories, we leverage two essential quantities that we compute in situ as an MD simulation evolves: the largest eigenvalue of alpha-Carbon (C𝛼) distance matrices (LEV) capturing molecular states and the effective sample size (ESS) identifying fast transitions of molecular states. 

To start the loading process we set the source of these trajectories by using the source() function of Pyemma's library and use the same features as for the full trajectories

In [ ]:
mdtraj_trimmed_trajectory_1 = coor.source([trimmed_trajectory_1], features=feat)
mdtraj_trimmed_trajectory_2 = coor.source([trimmed_trajectory_2], features=feat)

We load the trajectories as MDtraj objects

In [ ]:
mdtrajectories_trimmed_1 = md.load(trimmed_trajectory_1, top=top_file)
mdtrajectories_trimmed_2 = md.load(trimmed_trajectory_2, top=top_file)

Finally we map all input data as a list of arrays

In [ ]:
loaded_trajs_1= mdtraj_trimmed_trajectory_1.get_output()
loaded_trajs_2 = mdtraj_trimmed_trajectory_2.get_output()

## Calculating tICA for trimmed trajectories using the full tICA object

Since we have the tICA object of the full trajectories, we map the trimmed trajectories into the same space, to that end we use the transform() function of Pyemma

In [ ]:
tica_transformed_trim_1 = tica_obj.transform(loaded_trajs_1)
tica_transformed_trim_2 = tica_obj.transform(loaded_trajs_2)

As we did with the full trajectories, we concatenate the trimmed trajectories in a single variable.

In [ ]:
tica_concatenated_trim_1 = np.concatenate(tica_transformed_trim_1)
tica_concatenated_trim_2 = np.concatenate(tica_transformed_trim_2)

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.ticker as tick

viridis = cm.get_cmap("viridis", 256)
shadow = viridis(np.linspace(0, 1, 1))
shadow[0] = np.array([150 / 256, 150 / 256, 150 / 256, 1])  # Grey
newcmp = ListedColormap(shadow)
logscale = True
color_plot = viridis
color_plot = "nipy_spectral"


# fig, ax = plt.subplots(figsize=(12,7))
f, (ax1, ax2, ax3) = plt.subplots(ncols=3, sharey=True, sharex=True, figsize=(14, 8))

##------------------------------FULL--------------------------------
# Full tICA space
actualfig, _, misc, min_v, max_v, levels_v = jpccck.plot_density(
    tica_concatenated_full[:, 0],
    tica_concatenated_full[:, 1],
    cmap=color_plot,
    # levels=5,
    zorder=1,
    # cbar=False,
    cbar_orientation="horizontal",
    nbins=300,
    logscale=logscale,
    ax=ax1,
    cbar_label=full_title
)
# cbar_ = f.colorbar(misc["mappable"], ax=[ax1, ax2, ax3])
# cbar_.set_label("Density")

##------------------------------LEV--------------------------------
# Full tICA space
jpccck.plot_density(
    tica_concatenated_full[:, 0],
    tica_concatenated_full[:, 1],
    cmap=newcmp,
    # levels=5,
    alpha=1,
    cbar=False,
    zorder=0,
    nbins=300,
    logscale=logscale,
    ax=ax2,
    
)

# LEV space
jpccck.plot_density(
    tica_concatenated_trim_1[:, 0],
    tica_concatenated_trim_1[:, 1],
    cmap=color_plot,
    # levels=5,
    zorder=1,
    # cbar=False,
    cbar_orientation="horizontal",
    nbins=300,
    logscale=logscale,
    ax=ax2,
    vmin=min_v,
    vmax=max_v,
    levels=levels_v,
    cbar_label=trimmed_1_title
)

##------------------------------ESS--------------------------------
# Full tICA space
jpccck.plot_density(
    tica_concatenated_full[:, 0],
    tica_concatenated_full[:, 1],
    cmap=newcmp,
    # levels=5,
    alpha=1,
    cbar=False,
    zorder=0,
    nbins=300,
    logscale=logscale,
    ax=ax3,
)

# ESS space
jpccck.plot_density(
    tica_concatenated_trim_2[:, 0],
    tica_concatenated_trim_2[:, 1],
    cmap=color_plot,
    # levels=5,
    zorder=1,
    # cbar=False,
    cbar_orientation="horizontal",
    nbins=300,
    logscale=logscale,
    ax=ax3,
    vmin=min_v,
    vmax=max_v,
    levels=levels_v,
    cbar_label=trimmed_2_title
)
plt.tight_layout()
if SAVE_PLOTS:
    plt.savefig(os.path.join(plot_output_folder, plots_names + "_density.png"))
plt.show()



## Clustering
Assigning trimmed data to the cluster centers of the full trajectory suing the euclidean distance

In [ ]:
dtrajs_trimmed_1 = coor.assign_to_centers(tica_concatenated_trim_1[:,:2], clust.clustercenters, metric='euclidean',return_dtrajs=True)
dtrajs_trimmed_2 = coor.assign_to_centers(tica_concatenated_trim_2[:,:2], clust.clustercenters, metric='euclidean',return_dtrajs=True)

## Estimating Markov Model

We build a Markov State Model (MSM) to describe the dynamics and kinetics of the Fs peptide folding. 

We create three MSM, one for the full trajectories, one for the trajectories trimmed using LEV and one for the trajectories trimmed using ESS.

We select a time lag of 100 and use the VAMP method

In [ ]:
M = pyemma.msm.estimate_markov_model(dtrajs, lag=100, score_method="VAMP1")
MT_1 = pyemma.msm.estimate_markov_model(dtrajs_trimmed_1, lag=100, score_method="VAMP1")
MT_2 = pyemma.msm.estimate_markov_model(dtrajs_trimmed_2, lag=100, score_method="VAMP1")

## Doing PCCA+

We perform another clustering method. Now we use the Perron-Cluster Clustering Analysis (PCCA) to do a fuzzy clustering.

Here we use the number of states defined at the beggining. The number of states were defined in our previous work.

In [ ]:

M.pcca(nstates)
MT_1.pcca(nstates)
MT_2.pcca(nstates)

We map the cluster assignments of the trimmed data to the metastable states and then get the metastable states.

During this task we use the same assignments of the full model because we want to map the trimmed points in the same clusters identified by the PCCA.

In [ ]:
metastable_traj = M.metastable_assignments[dtrajs_concatenated]
metastable_assignments_trimmed_1 = M.metastable_assignments[dtrajs_trimmed_1]
metastable_assignments_trimmed_2 = M.metastable_assignments[dtrajs_trimmed_2]

pcca_sets = M.metastable_sets
pcca_setsT = MT_1.metastable_sets
pcca_setsT_2 = MT_2.metastable_sets

## Plotting state maps
We use one of the functions from Pyemma to visualize the state maps that we got from doing the PCCA.

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(ncols=3, sharey=True, sharex=True, figsize=(18, 10))
cols = ["orange", "yellow", "limegreen", "cyan", "magenta", "royalblue"]
cols_2 = ["magenta", "royalblue", "cyan", "limegreen", "yellow", "orange"]
cmap = mpl.colors.ListedColormap(cols)
pyemma.plots.plot_state_map(
    tica_concatenated_full[:, 0],
    tica_concatenated_full[:, 1],
    metastable_traj,
    cbar_label="Full",
    cbar=True,
    cbar_orientation="horizontal",
    cmap=cmap,
    ncontours=200,
    nbins=200,
    ax=ax1,
)
ax1.set_xlabel("TIC 1")
ax1.set_ylabel("TIC 2")
pyemma.plots.plot_state_map(
    tica_concatenated_trim_1[:, 0],
    tica_concatenated_trim_1[:, 1],
    np.asarray(metastable_assignments_trimmed_1).reshape(-1),
    cbar_label="LEV",
    cbar=True,
    cbar_orientation="horizontal",
    cmap=cmap,
    ncontours=300,
    nbins=200,
    ax=ax2,
)
ax2.set_xlabel("TIC 1")
ax2.set_ylabel("TIC 2")
pyemma.plots.plot_state_map(
    tica_concatenated_trim_2[:, 0],
    tica_concatenated_trim_2[:, 1],
    np.asarray(metastable_assignments_trimmed_2).reshape(-1),
    cbar_label="ESS",
    cbar=True,
    cbar_orientation="horizontal",
    cmap=cmap,
    ncontours=300,
    nbins=200,
    ax=ax3,
)
ax3.set_xlabel("TIC 1")
ax3.set_ylabel("TIC 2")
ax1.grid()
ax2.grid()
ax3.grid()
plt.tight_layout()
if SAVE_PLOTS:
    plt.savefig(os.path.join(plot_output_folder, plots_names + "_state_map.png"))


# Calculating Energy

Now we calculate the Free Energy landscape for the three models we have (Full, LEV and ESS). We do this calculation using three different methods: umbrella sampling, Boltzmann equation and the method provided by Pyemma to do this calculation.

## Stationary Distribution - Energy


This cell calculates the free energies for three different distributions: full, trimmed, and trimmed ess.
It uses the Boltzmann equation based on stationary distributions and total distributions of each distribution to calculate the probabilities (pi).
The free energies are then calculated as the negative logarithm of the probabilities.
The minimum value of the free energies for the full distribution is subtracted from all the free energies.


In [ ]:

# Full
stationary_distribution = M.stationary_distribution
total_distribution = float(stationary_distribution.sum())
pi = stationary_distribution / total_distribution
free_energies = -np.log(pi)
min_full = np.min(free_energies)
free_energies -= np.min(free_energies)
free_energies = np.round(free_energies, 2)

# LEV
stationary_distributionT_1 = MT_1.stationary_distribution
total_distributionT_1 = float(stationary_distributionT_1.sum())
piT = stationary_distributionT_1 / total_distributionT_1
free_energiesT_1 = -np.log(piT)
free_energiesT_1 -= min_full
free_energiesT_1 -= np.min(free_energiesT_1)
free_energiesT_1 = np.round(free_energiesT_1, 2)

# ESS
stationary_distributionT_2 = MT_2.stationary_distribution
total_distributionT_2 = float(stationary_distributionT_2.sum())
piT_2 = stationary_distributionT_2 / total_distributionT_2
free_energiesT_2 = -np.log(piT_2)
free_energiesT_2 -= min_full
free_energiesT_2 -= np.min(free_energiesT_2)
free_energiesT_2 = np.round(free_energiesT_2, 2)

## Finding Frames closest to minimum free energy
To find the most representatives structures per state, we find the closest frames to the minimum values per state.

We find the 10 closest frames, so we can compare them to verify whether their structure is similar or not.

We do this for every model (Full, LEV and ESS) and Free Energy calculation method (umbrella sampling, Boltzmann equation and Pyemma's)



In [ ]:
## TODO: change to dataframe structure or create a class with these attributes so we only pass the object
print("Stationary")
(
    frames_closest_to_minimum_energy_coor_stationary,
    frames_closest_to_minimum_energy_coor_T_1_stationary,
    frames_closest_to_minimum_energy_coor_T_2_stationary,
    frames_closest_to_minimum_energy_stationary,
    frames_closest_to_minimum_energyT_1_stationary,
    frames_closest_to_minimum_energyT_2_stationary,
    frames_10_closest_to_minimum_energy_stationary,
    frames_10_closest_to_minimum_energyT_1_stationary,
    frames_10_closest_to_minimum_energyT_2_stationary,
    minimum_energy_stationary,
    minimum_energyT_stationary,
    minimum_energyT_2_stationary,
) = jpccck.find_frames_closest_to_minimum_energy(
    FES=free_energies,
    FES_T=free_energiesT_1,
    FES_T_2=free_energiesT_2,
    M=M,
    MT=MT_1,
    MT_2=MT_2,
    clust=clust,
    tica_concatenated_full=tica_concatenated_full,
    tica_concatenated_trim_1=tica_concatenated_trim_1,
    tica_concatenated_trim_2=tica_concatenated_trim_2,
    nstates=nstates,
)

## Energy Ranking - Stationary

In [ ]:
min_coor_full = list(np.round(frames_closest_to_minimum_energy_coor_stationary, 1))
min_coor_1 = list(np.round(frames_closest_to_minimum_energy_coor_T_1_stationary, 1))
min_coor_2 = list(np.round(frames_closest_to_minimum_energy_coor_T_2_stationary, 1))

data = {
    'State': range(0, len(minimum_energy_stationary)),
    f'Coordinates {full_title}': min_coor_full,
    f'Minimum Energy ({full_title})': minimum_energy_stationary,
    f'Coordinates {trimmed_1_title}': min_coor_1,
    f'Minimum Energy ({trimmed_1_title})': minimum_energyT_stationary,
    f'Coordinates {trimmed_2_title}': min_coor_2,
    f'Minimum Energy ({trimmed_2_title})': minimum_energyT_2_stationary,
}

df = pd.DataFrame(data)
df_sorted = df.sort_values(by=f'Minimum Energy ({full_title})')
display(df_sorted)
if SAVE_PLOTS:
    # save table as image
    dfi.export(df_sorted, os.path.join(plot_output_folder, plots_names + "_minimum_energy_table.png"),table_conversion = 'matplotlib' )
    df.to_csv(f'{plot_output_folder}/{plots_names}_minimum_energy_table.csv', mode='a', header=True, index=False)


## Saving closest frames to disk as pdb files

We save the closest frames to disk to visualize them using VMD and to do the RMSD calculation later.

The folder that contains the frames will have the 10 closest frames but also the closest one. We name this folder with the datetime.

In [ ]:
## TODO: change to dataframe structure or create a class with these attributes so we only pass the object
if SAVE_FRAMES:
    folder = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
    folder_path = f"{frames_closest_folder}{folder}"
    os.mkdir(folder_path)
    folder_path_10 = f"{folder_path}/frames_10"
    os.mkdir(folder_path_10)

    frames_10_total_stationary, frames_10_files_stationary = jpccck.save_10_frames(
        folder_path_10,
        "stationary",
        frames_10_closest_to_minimum_energy_stationary,
        mdtrajectories,
        "full",
    )

    frames_10T_1_total_stationary, frames_10_filesT_1_stationary = (
        jpccck.save_10_frames(
            folder_path_10,
            "stationary",
            frames_10_closest_to_minimum_energyT_1_stationary,
            mdtrajectories_trimmed_1,
            "trim_1",
        )
    )

    frames_10T_2_total_stationary, frames_10_filesT_2_stationary = (
        jpccck.save_10_frames(
            folder_path_10,
            "stationary",
            frames_10_closest_to_minimum_energyT_2_stationary,
            mdtrajectories_trimmed_2,
            "trim_2",
        )
    )

    frames_minimum_stationary, frames_minimum_files_stationary = jpccck.save_frames(
        folder_path,
        "stationary",
        frames_closest_to_minimum_energy_stationary,
        mdtrajectories,
        "full",
    )

    frames_minimumT_1_stationary, frames_minimum_filesT_1_stationary = (
        jpccck.save_frames(
            folder_path,
            "stationary",
            frames_closest_to_minimum_energyT_1_stationary,
            mdtrajectories_trimmed_1,
            "trim_1",
        )
    )

    frames_minimumT_2_stationary, frames_minimum_filesT_2_stationary = (
        jpccck.save_frames(
            folder_path,
            "stationary",
            frames_closest_to_minimum_energyT_2_stationary,
            mdtrajectories_trimmed_2,
            "trim_2",
        )
    )

## Visualizing Free energy landscapes and minimum per state
We use the tricontour plot to visualize the Free Energy landscape for all the three methods we used. In this plot we also present the cluster centers that we got from the kmeans clustering, in every color we also see the different states and as a white star we see the minimum Free Energy points

In [ ]:
jpccck.plot_free_energy(
    cc_x=cc_x,
    cc_y=cc_y,
    free_energy_per_cluster=free_energies,
    free_energy_per_clusterT=free_energiesT_1,
    free_energy_per_clusterT_2=free_energiesT_2,
    dist_cmap=dist_cmap,
    # title="Energy - Stationary Distribution",
    nstates=nstates,
    pcca_sets=pcca_sets,
    cols=cols,
    size=size,
    frames_closest_to_minimum_energy_coor=frames_closest_to_minimum_energy_coor_stationary,
    frames_closest_to_minimum_energy_coorT=frames_closest_to_minimum_energy_coor_T_1_stationary,
    frames_closest_to_minimum_energy_coorT_2=frames_closest_to_minimum_energy_coor_T_2_stationary,
    no_points=True,
    full_title=full_title,
    trimmed_1_title=trimmed_1_title,
    trimmed_2_title=trimmed_2_title,
)
if SAVE_PLOTS:
    plt.savefig(os.path.join(plot_output_folder, plots_names + "_energy.png"))


In [ ]:
jpccck.plot_free_energy(
    cc_x=cc_x,
    cc_y=cc_y,
    free_energy_per_cluster=free_energies,
    free_energy_per_clusterT=free_energiesT_1,
    free_energy_per_clusterT_2=free_energiesT_2,
    dist_cmap=dist_cmap,
    # title="Energy - Stationary Distribution",
    nstates=nstates,
    pcca_sets=pcca_sets,
    cols=cols,
    size=35,
    frames_closest_to_minimum_energy_coor=frames_closest_to_minimum_energy_coor_stationary,
    frames_closest_to_minimum_energy_coorT=frames_closest_to_minimum_energy_coor_T_1_stationary,
    frames_closest_to_minimum_energy_coorT_2=frames_closest_to_minimum_energy_coor_T_2_stationary,
    no_points=False,
    full_title=full_title,
    trimmed_1_title=trimmed_1_title,
    trimmed_2_title=trimmed_2_title,
)
if SAVE_PLOTS:
    plt.savefig(os.path.join(plot_output_folder, plots_names + "_energy.png"))


# Calculating RMSD average within same states (10 Closest Frames)
We take the 10 closest frames and calculate the RMSD average among them. We are using two methods to do this. The mdtraj method aligns all the frames and then does the comparison. The Compute method does the comparison without aligning the frames. For both methods we use the closest frame to the minium Free Energy point as reference. We do this only for Pyemma's Free Energy due to this one represented better the landscape comparing it to the full simulation

In [ ]:
data_average_rmsd = {'State': range(0, len(minimum_energy_stationary))}
data_average_rmsd["full_mdtraj"] = jpccck.calculate_average_rmsd(frames_10_total_stationary, frames_10_files_stationary, 'mdtraj', nstates, selection)

data_average_rmsd["lev_mdtraj"] = jpccck.calculate_average_rmsd(frames_10T_1_total_stationary, frames_10_filesT_1_stationary, 'mdtraj', nstates, selection)

data_average_rmsd["ess_mdtraj"] = jpccck.calculate_average_rmsd(frames_10T_2_total_stationary, frames_10_filesT_2_stationary, 'mdtraj', nstates, selection)
df_average_rmsd = pd.DataFrame(data_average_rmsd)
display(df_average_rmsd)



In [ ]:
import matplotlib.pyplot as plt

# First plot: Free Energies and Free EnergiesT
df1 = pd.DataFrame({full_title: free_energies, trimmed_1_title: free_energiesT_1})
pl1 = df1.plot(figsize=(12, 6))
plt.xlabel('Microstate', fontsize=14)
plt.ylabel('Free Energy', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=12)  # Increase the font size of the ticks on both axes
plt.title('FES', fontsize=14)
plt.legend(fontsize=14)
plt.savefig(os.path.join(plot_output_folder, plots_names + "_free_energy_line_comparison_1.png"))
plt.show()

# Second plot: Free Energies and Free EnergiesT Ess
df2 = pd.DataFrame({full_title: free_energies, trimmed_2_title: free_energiesT_2})
pl2 = df2.plot(figsize=(12, 6))
plt.xlabel('Microstate', fontsize=14)
plt.ylabel('Free Energy', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=12)  # Increase the font size of the ticks on both axes
plt.title('FES', fontsize=14)
plt.legend(fontsize=14)
if SAVE_PLOTS:
    plt.savefig(os.path.join(plot_output_folder, plots_names + "_free_energyT_2_line_comparison_2.png"))
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
# Create a figure with subplots
fig, axs = plt.subplots(4, 2, figsize=(12, 15))

for state in range(nstates):
    # Gets the indexes of the cluster centers in the state
    indexes_cluster_centers_per_state = pcca_sets[state]
    indexes_cluster_centers_per_stateT = pcca_sets[state]
    indexes_cluster_centers_per_stateT_2 = pcca_sets[state]
    # Gets the coordinates of the cluster centers in the state
    coord_cluster_per_state = clust.clustercenters[
        indexes_cluster_centers_per_state
    ]
    coord_cluster_per_stateT = clust.clustercenters[
        indexes_cluster_centers_per_stateT
    ]
    coord_cluster_per_stateT_2 = clust.clustercenters[
        indexes_cluster_centers_per_stateT_2
    ]
    # Gets the free energy of the cluster centers in the state
    energy_of_cluster_center_per_state = free_energies[indexes_cluster_centers_per_state]
    energy_of_cluster_center_per_stateT = free_energiesT_1[indexes_cluster_centers_per_stateT]
    energy_of_cluster_center_per_stateT_2 = free_energiesT_2[
        indexes_cluster_centers_per_stateT_2
    ]

    # Smooth curve for full
    x = range(len(energy_of_cluster_center_per_state))
    x1 = range(len(energy_of_cluster_center_per_stateT))
    x2 = range(len(energy_of_cluster_center_per_stateT_2))
    y = energy_of_cluster_center_per_state
    y1 = energy_of_cluster_center_per_stateT
    y2 = energy_of_cluster_center_per_stateT_2
    
    X_Y_Spline = make_interp_spline(x, y)
    X_Y_Spline1 = make_interp_spline(x1, y1)
    X_Y_Spline2 = make_interp_spline(x2, y2)

    # Returns evenly spaced numbers over a specified interval.
    X_ = np.linspace(min(x), max(x), 500)
    X1_ = np.linspace(min(x1), max(x1), 500)
    X2_ = np.linspace(min(x2), max(x2), 500)
    Y_ = X_Y_Spline(X_)
    Y1_ = X_Y_Spline1(X1_)
    Y2_ = X_Y_Spline2(X2_)

    # Plotting the Graph in the corresponding subplot
    row = state // 2
    col = state % 2
    axs[row, col].plot(X_, Y_, label=full_title)
    axs[row, col].plot(X1_, Y1_, label=trimmed_1_title)
    axs[row, col].plot(X2_, Y2_, label=trimmed_2_title)
    axs[row, col].set_title(f'Free Energy for State {state}',fontsize=14)
    axs[row, col].tick_params(axis='both', which='major', labelsize=12)  # Increase the font size of the ticks on both axes
    axs[row, col].set_xlabel('Cluster Center Index',fontsize=14)
    axs[row, col].set_ylabel('Free Energy k/T',fontsize=14)
    # axs[row, col].set_ylim(-0.5)  # Set the lower limit of the y-axis to zero
    # set the highest limit in y-axis to 5.2
    axs[row, col].set_ylim(-0.5, 5.2)
    axs[row, col].legend(fontsize=14)
    axs[row, col].axhline(y=0, color='blue', linestyle='dotted')
    axs[row, col].grid()
    if state == 5:
        axs[row +1, col-1].plot(X_, Y_, label=full_title)
        axs[row +1, col-1].plot(X1_, Y1_, label=trimmed_1_title)
        axs[row +1, col-1].tick_params(axis='both', which='major', labelsize=12)  # Increase the font size of the ticks on both axes
        axs[row +1, col-1].set_title(f'Free Energy for State {state}', fontsize=14)
        axs[row +1, col-1].set_xlabel('Cluster Center Index', fontsize=14)
        axs[row +1, col-1].set_ylabel('Free Energy k/T', fontsize=14)
        axs[row +1, col-1].set_ylim(-0.5)  # Set the lower limit of the y-axis to zero
        axs[row +1, col-1].legend(fontsize=14)
        axs[row+1, col-1].set_ylim(-0.5, 5.2)
        axs[row +1, col-1].axhline(y=0, color='blue', linestyle='dotted')
        axs[row +1, col-1].grid()

        axs[row +1, col].plot(X_, Y_, label=full_title)
        axs[row +1, col].plot(X2_, Y2_, label=trimmed_2_title)
        axs[row +1, col].tick_params(axis='both', which='major', labelsize=12)  # Increase the font size of the ticks on both axes
        axs[row +1, col].set_title(f'Free Energy for State {state}',fontsize=14)
        axs[row +1, col].set_xlabel('Cluster Center Index',fontsize=14)
        axs[row +1, col].set_ylabel('Free Energy k/T',fontsize=14)
        axs[row +1, col].set_ylim(-0.5)  # Set the lower limit of the y-axis to zero
        axs[row +1, col].legend(fontsize=14)
        axs[row+1, col].set_ylim(-0.5, 5.2)
        axs[row +1, col].axhline(y=0, color='blue', linestyle='dotted')
        axs[row +1, col].grid()

# Adjust the spacing between subplots
plt.tight_layout()
if SAVE_PLOTS:
    plt.savefig(os.path.join(plot_output_folder, plots_names + "_free_energy_comparison_per_state.png"))
# Display the figure
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Compute the mean absolute difference
mean_abs_diff = np.mean(np.abs(free_energies - free_energiesT_1))
print(f"Mean absolute difference between the free energies of the full and trimmed (LEV) models: {mean_abs_diff}")
mean_abs_diff_2 = np.mean(np.abs(free_energies - free_energiesT_2))
print(f"Mean absolute difference between the free energies of the full and trimmed (ESS) models: {mean_abs_diff_2}")

lev_folder = os.path.join(plot_output_folder, plots_names + "_1_MAD")
print(lev_folder)

first_name = (plots_1_name.split("termination_1_")[-1])
second_name = (plots_2_name.split("_1_")[-1])

# Create a dictionary with the data
data_mad = {
    'Mean_abs_diff': [mean_abs_diff, mean_abs_diff_2],
    'Percentage': [first_name, second_name]
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data_mad)

if SAVE_PLOTS:
    # Save the DataFrame to a CSV file (append mode)
    df.to_csv(f'{plot_output_folder}/mean_absolute_values.csv', mode='a', header=False, index=False)





In [ ]:
# Read the DataFrame from the CSV file
df = pd.read_csv(f'{plot_output_folder}/lev/mean_absolute_values.csv')

# Print the DataFrame
print(df)

In [ ]:
plot = df.plot(y='Mean_abs_diff', kind='line')
plot.set_xticks(range(len(df)))
plot.set_xticklabels(df['Percentage'], rotation=60, fontsize=12)
plot.axhline(y=0, color='blue', linestyle='dotted')
plot.set_ylabel('Mean Absolute Difference', fontsize=14)
plot.set_xlabel('Percentage', fontsize=14)
plot.scatter(range(len(df)), df['Mean_abs_diff'], color='red', marker='o')
plot.set_title('Full vs LEV trimmed models', fontsize=14)
plot.grid()
if SAVE_PLOTS:
    plt.savefig(f'{plot_output_folder}/lev/LEV_mean_absolute_values.png')
plt.show()


## HMM
We do the estimation of the Hidden Markov Model that can represent better the transitions among states. 

In [ ]:
hmm_full = pyemma.msm.estimate_hidden_markov_model(dtrajs, 6, lag=100)
hmm_1 = pyemma.msm.estimate_hidden_markov_model(dtrajs_trimmed_1, 6, lag=100)
hmm_2 = pyemma.msm.estimate_hidden_markov_model(dtrajs_trimmed_2, 6, lag=100)

## Plotting the state maps for the HMM

In [ ]:
cols = ["orange", "yellow", "limegreen", "cyan", "royalblue", "magenta"]
# colsT = ["limegreen", "yellow", "orange", "magenta", "cyan", "royalblue"]
cmap = mpl.colors.ListedColormap(cols)
fig, (ax1,ax2,ax3) = plt.subplots(ncols=3, figsize=(15, 10), sharex=True, sharey=True)


pyemma.plots.plot_state_map(
    *tica_concatenated_full[:, :2].T,
    hmm_full.metastable_assignments[dtrajs_concatenated],
    ax=ax1,
    cmap='viridis',
    nbins=300,
    cbar=True,
    cbar_orientation="horizontal",
)
ax1.set_xlabel("TIC 1")
ax1.set_ylabel("TIC 2")
pyemma.plots.plot_state_map(
    *tica_concatenated_trim_1[:, :2].T,
    np.asarray(hmm_full.metastable_assignments[dtrajs_trimmed_1]).reshape(-1),
    ax=ax2,
    cmap='viridis',
    nbins=300,
    cbar=True,
    cbar_orientation="horizontal",
)
ax2.set_xlabel("TIC 1")
ax2.set_ylabel("TIC 2")
pyemma.plots.plot_state_map(
    *tica_concatenated_trim_2[:, :2].T,
    np.asarray(hmm_full.metastable_assignments[dtrajs_trimmed_2]).reshape(-1),
    ax=ax3,
    cmap='viridis',
    nbins=300,
    cbar=True,
    cbar_orientation="horizontal",
)
ax3.set_xlabel("TIC 1")
ax3.set_ylabel("TIC 2")
ax1.grid()
ax2.grid()
ax3.grid()
plt.tight_layout()
if SAVE_PLOTS:
    plt.savefig(os.path.join(plot_output_folder, plots_names + "_hmm.png"))


## Calculating Free Energy - HMM - Stationary
We calculate the Free Energy of the HMM. This calculation uses same Boltzmann equation we previously used.

In [ ]:
# full
stationary_distribution = hmm_full.stationary_distribution
total_distribution = float(stationary_distribution.sum())
pi = stationary_distribution / total_distribution
hmm_full_free_energies = -np.log(pi)
min_full = np.min(hmm_full_free_energies)
hmm_full_free_energies -= np.min(hmm_full_free_energies)

# trimmed - LEV
stationary_distributionT = hmm_1.stationary_distribution
total_distributionT = float(stationary_distributionT.sum())
piT = stationary_distributionT / total_distributionT
hmm_1_free_energiesT = -np.log(piT)
hmm_1_free_energiesT -= min_full
hmm_1_free_energiesT -= np.min(hmm_1_free_energiesT)

# trimmed - ESS
stationary_distributionT_2 = hmm_2.stationary_distribution
total_distributionT_2 = float(stationary_distributionT_2.sum())
piT_2 = stationary_distributionT_2 / total_distributionT_2
hmm_2_free_energiesT_2 = -np.log(piT_2)
hmm_2_free_energiesT_2 -= min_full
hmm_2_free_energiesT_2 -= np.min(hmm_2_free_energiesT_2)

In [ ]:
data[f'hmm_{full_title}'] = hmm_full_free_energies
data[f'hmm_{lev_title}'] = hmm_1_free_energiesT
data[f'hmm_{ess_title}'] = hmm_2_free_energiesT_2
df = pd.DataFrame(data)
# df_sorted = df.sort_values(by='Minimum Energy (PyEmma)')
display(df)
if SAVE_PLOTS:
    # save table as image
    dfi.export(df, os.path.join(plot_output_folder, plots_names + "_hmm_free_energy_table.png"),table_conversion = 'matplotlib' )
